In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

if os.getcwd().endswith("nb"):
    os.chdir("..")  # should be bone-age repo

In [ ]:
%%bash
CKPT_DIR="$BASE_DIR/output/best_models/"  # path to checkpoint directory, TODO change if needed
BASE_DIR=$(pwd)
OUTPUT_DIR="$BASE_DIR/output/predictions_rsna/"  # path in which all prediction .csv files will be saved

l=$(cat <<EOF
masked_effnet_supShal_highRes_fancy_aug.ckpt
masked_effnet_super_shallow_fancy_aug.ckpt
masked_effnet-b4_shallow_pretr_fancy_aug.ckpt
EOF
)
mkdir -p $OUTPUT_DIR

for c in $l
do
  echo "predicting with model $c"
  python predict.py \
        --ckp_path="$CKPT_DIR/$c" \
        --legacy_ckp \
        --gpus=1 \
        --precision=16 \
        --num_workers=8 \
        --batch_size=32 \
        --mask_crop_size=-1 \
        --mask_dirs=../data/masks/fscnn_cos \
        --img_dir=../data/annotated \
        --annotation_csv=data/rsna_test.csv \
        --output_path="$OUTPUT_DIR/${c%.ckpt}.csv" \

done

Here, the `annotation_csv` argument should point to the .csv file containing at least the `image_ID` (matching the file name), the subdirectory within the `img_dir` directory containing the files (might be left blank), and the `sex` (females encoded as 0. males at 1). To subset the used images, please refer to the splitting arguments in the source code.

Make sure that:
  * `mask_dirs` points to the directory containing the masks (usually the fscnn_cos directory, see hand-segmentation repo)
  * `legacy_ckp` is selected for old (i.e. non-mtl) checkpoints (basically all relevant ones)
  * `mask_crop_size` is set appropriately (i.e. -1 = no cropping for RSNA data, for external data 1.15 might be better to compensate for weird scanning)
  * `img_dir` points to the directory containing the images (usually the 'annotated' directory, see the data-management repo)
  * `gpus`, `batch_size`, and `num_workers` are set appropriately depending on the hardware

In [3]:
# needs to match $OUTPUT_DIR in the bash script above
output_dir = "output/predictions_rsna"
anno = pd.read_csv("data/rsna_test.csv")

# gather all predictions
df = {c.replace(".csv", ""): pd.read_csv(f"{output_dir}/{c}")[["image_ID", "pred_bone_age"]].rename(columns={"pred_bone_age": c.replace(".csv", "")}) for c in os.listdir(output_dir) if not "ensemble" in c}
result = pd.concat([df[c].set_index('image_ID') for c in df], axis=1)

# take mean of all predictions for ensemble
result["ensemble_bone_age"] = result.mean(axis=1)

# add ground truth if available
result = result.reset_index().merge(anno[["image_ID", "bone_age"]].rename(columns={"bone_age": "gt_bone_age"}))

# save to csv
result.to_csv(f"{output_dir}/ensemble.csv", index=False)

# sanity check MAD score
np.linalg.norm(result["ensemble_bone_age"] - result["gt_bone_age"], 1) / len(result)

3.8705049999999996